# NVCL Dataservices Notebook - Get Datasets

This notebook demonstrates on how you can query NVCL Dataservices to obtain the list of datasets available

Webserver URL of different states:

https://nvclwebservices.csiro.au/geoserver/wfs <br/>
https://gs.geoscience.nsw.gov.au/geoserver/wfs <br/>
https://geology.information.qld.gov.au/geoserver/wfs <br/>
https://sarigdata.pir.sa.gov.au/geoserver/wfs <br/>
http://geossdi.dmp.wa.gov.au/services/wfs <br/>
https://www.mrt.tas.gov.au/web-services/wfs <br/>
http://geology.data.nt.gov.au/geoserver/wfs <br/>
http://geology.data.vic.gov.au/nvcl/wfs <br/>

In [1]:
from nvcl_kit.reader import NVCLReader 
from types import SimpleNamespace
param = SimpleNamespace()

param.WFS_URL = "http://geology.data.vic.gov.au/nvcl/wfs"

# URL of NVCL service
param.NVCL_URL = "http://geology.data.vic.gov.au/NVCLDataServices/"

reader = NVCLReader(param)

In [2]:
if not reader.wfs:
    print("ERROR!")

In [3]:
nvcl_id_list = reader.get_nvcl_id_list()

In [8]:
import pandas as pd
import xmltodict
import json

data = []
for nvcl_id in nvcl_id_list:
  response_str = reader.svc.get_dataset_collection(nvcl_id)
  dataset_obj = xmltodict.parse(response_str)["DatasetCollection"]['Dataset']
  for dataset_dict in (dataset_obj if type(dataset_obj) is list else [dataset_obj]):
    dataset_dict['nvclID'] = nvcl_id
    data.append(dataset_dict)

dataset_df = pd.DataFrame(data)

In [84]:
from ipyaggrid import Grid

dataset_view_df = dataset_df[['nvclID', 'DatasetID', 'boreholeURI', 'DatasetName']]
column_defs = [
    {'field': 'nvclID', 'headerName': 'NVCL Id', 'headerCheckboxSelection': True, 'checkboxSelection': True},
    {'field': 'DatasetID', 'headerName': 'Dataset Id'},
    {'field': 'DatasetName', 'headerName': 'Dataset Name'},
    {'field': 'boreholeURI', 'headerName': 'Borehole URI'}
]
grid_options = {
    'columnDefs' : column_defs,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'rowSelection': 'multiple'
}

# function isFirstColumn(params) {
#     var displayedColumns = params.columnApi.getAllDisplayedColumns();
#     var thisIsFirstColumn = displayedColumns[0] === params.column;
#     return thisIsFirstColumn;
# }

dataset_g = Grid(grid_data=dataset_df,
        height=400,
        theme='ag-theme-balham',
        grid_options=grid_options,
        columns_fit='auto',
        quick_filter=True,
        export_mode='buttons')
dataset_g

Grid(columns_fit='auto', compress_data=True, export_mode='buttons', height='400px', menu={'buttons': [{'name':…

In [91]:
if 'rows' not in dataset_g.grid_data_out:
    print ('Please select your datasets and export rows')

In [94]:
selected_datasets_df = dataset_g.grid_data_out['rows']

for selected_dataset_id in selected_datasets_df['DatasetID']:
    response = reader.svc.download_tsg(selected_dataset_id, 'venkataramanan.selvaraju@csiro.au')

In [95]:
response = reader.svc.get_download_tsg_status('venkataramanan.selvaraju@csiro.au')

response

b'\r\n\r\n\r\n\r\n<html>\r\n  <head>\r\n\t\t<title>NVCL Download Services : TSG file Download Requests Status Page</title>\r\n\t\t<link rel="stylesheet" href="style/style.css" type="text/css"/>\r\n  </head>\r\n  <body class="NVCLDSBody">\r\n    <h2 class="NVCLDSh2">NVCL Download Services : TSG file Download Requests Status Page</h2>\r\n\r\n\t<table class="NVCLDSTable">\r\n\t<thead>\r\n\t<tr><th>Timestamp</th><th>Status</th><th>Details</th><th></th></tr>\r\n\t</thead>\r\n\t<tbody>\r\n\t\r\n\t\t\t<tr class="">\r\n\t\t\t\t<td>Mar 2, 2020 4:28:02 PM</td>\r\n\t\t\t\t<td>Queued</td>\r\n\t\t\t\t<td>Request for dataset with ID: c7bfb49c-2e85-4d31-97f1-e9e273e299d</td>\r\n\t\t\t\t<td></td>\r\n\t\t\t</tr>\r\n\t\t\r\n\t\t\r\n\t</tbody>\r\n    </table>\r\n\r\n  </body>\r\n</html>'

In [9]:
from ipywidgets import Dropdown, Layout, HTML
from IPython.display import clear_output

dataset_dw = Dropdown(
    options = list(zip(dataset_df['DatasetName'], dataset_df['DatasetID'])),
    value = dataset_df['DatasetID'][0],
    description = 'Dataset',
    layout = Layout(margin='20px')
)

imglogs = reader.get_tray_thumb_imglogs(dataset_df['DatasetID'][0])
mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)

def on_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    imglogs = reader.get_tray_thumb_imglogs(change['new'])
    mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)
    clear_output(True)
    display(dataset_dw, HTML(value=mosaic_html))

dataset_dw.observe(on_change)

display(dataset_dw, HTML(value=mosaic_html))

Dropdown(description='Dataset', layout=Layout(margin='20px'), options=(('877270_KIND168', 'c7bfb49c-2e85-4d31-…

HTML(value='\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n  <head>\r\n    <title>NVCL Data Services :: …